# Bird Species Classifier for AML project using Keras/TensorFlow
## University of Vienna, SS 2022

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

# To search directories
import os
import glob

# To visualize data
import PIL
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('darkgrid')

In [ ]:
print("TensorFlow Version: ", tf.__version__)
print("Keras Version: ", keras.__version__)
print("GPU devices: ", tf.config.list_physical_devices('gpu'))

# 1. Examine and understand data
## CSV data
The ``birds.csv`` contains information of the dataset. Let's look into the structure of the data.

In [ ]:
# Create a dataframe from the csv
birds_df = pd.read_csv("../input/100-bird-species/birds.csv")
# clean column names
birds_df.columns = [col.replace(' ', '_').lower() for col in birds_df.columns]
birds_df.head()

In [ ]:
birds_df.info()

In [ ]:
birds_df.value_counts("data_set").head()

In [ ]:
# Frequency of bird species in the whole dataset
print("|species | f|")
birds_df.value_counts("class_index")

In [ ]:
# Look at csv entries for one single bird

#mask = birds_df['labels'].str.contains("ABBOTTS BABBLER") # Search for text fragment
#mask = birds_df.query('labels == "ABBOTTS BABBLER"') # query for name (case sensitive!)
mask = birds_df.loc[birds_df['class_index'] == 0]
print(mask.value_counts("data_set"))
mask

## Image data

In [ ]:
# File directories
root_dir = "../input/100-bird-species"
train_dir = "../input/100-bird-species/train"
valid_dir = "../input/100-bird-species/valid"
test_dir = "../input/100-bird-species/test"

### Plot a bird image

In [ ]:
def showFirstBird(bird_name="MALLARD DUCK"):
    """
    Print out file paths of images in the valid_dir and show the first image of a given species.
    """
    import glob
    img_files = []
    for img in glob.glob(os.path.join(valid_dir, bird_name)+"/*"):
        img_files.append(img)
        
    for i in img_files:
        print(i) # Print file path
        ifile = tf.io.read_file(i) # Reads the contents of file
        img_dec = tf.io.decode_image(ifile) # Decodes an image file
        print("File shape: ", img_dec.shape, "\n")
        
    img = PIL.Image.open(str(img_files[0]))
    return img
    
showFirstBird()

# 2. Create a dataset for the model
## Generate tf.data.Dataset objects from a directory
Take image files from a directory on disk and generate a ``tf.data.Dataset`` for train, validation and test dataset. ``image_dataset_from_directory()`` is a special TensorFlow data generator function.

In [ ]:
"""
Achieving peak performance requires an efficient input pipeline that delivers data for 
the next step before the current step has finished. The tf.data API helps to build flexible 
and efficient input pipelines.
~ https://www.tensorflow.org/guide/data_performance
"""
IMAGE_SIZE=(150,150) # original size: 224,224 # Resolution decreased to speed up training time
BATCH_SIZE=32 # default=32
SEED=42
np.random.seed(42)
tf.random.set_seed(42)

train_data = image_dataset_from_directory(
    directory=train_dir,
    validation_split=0.5,
    label_mode='categorical',
    batch_size=BATCH_SIZE, 
    image_size=IMAGE_SIZE,
    subset='training',
    seed=SEED,
    shuffle=True   # default
)
class_names = train_data.class_names
num_classes = len(class_names)
#print("Class names: ", class_names[:5])

valid_data = image_dataset_from_directory(
    directory=valid_dir,
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=SEED,
    shuffle=True   # default
)

test_data = image_dataset_from_directory(
    directory=test_dir,
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=SEED,
    shuffle=False
)
len(train_data)

## Configure the dataset for performance
To prevent I/O blocking while retrieving data from disk we use buffered prefetching.  

The tf.data API provides the tf.data.Dataset.prefetch transformation. It can be used 
to decouple the time when data is produced from the time when data is consumed. In particular, 
the transformation uses a background thread and an internal buffer to prefetch elements from 
the input dataset ahead of the time they are requested. The number of elements to prefetch 
should be equal to (or possibly greater than) the number of batches consumed by a single training step. 
You could either manually tune this value, or set it to tf.data.AUTOTUNE, which will prompt the 
tf.data runtime to tune the value dynamically at runtime.

 ~ https://www.tensorflow.org/guide/data_performance

In [ ]:
train_data_pf = train_data.prefetch(buffer_size = tf.data.AUTOTUNE)
valid_data_pf = valid_data.prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_pf = test_data.prefetch(buffer_size = tf.data.AUTOTUNE)

# 3. Create Model 

## Define model architecture

In [ ]:
INPUT_SHAPE=(150, 150, 3)

model = tf.keras.models.Sequential([
    
    # handy input layer
    layers.Input(INPUT_SHAPE),
    
    # convolution layers
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.BatchNormalization(),    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.BatchNormalization(),
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.BatchNormalization(),
    layers.Conv2D(8, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.BatchNormalization(),
    
    # Last fully-connected layer
    layers.Flatten(input_shape=INPUT_SHAPE),
    #layers.Dropout(0.2), # higher -> more regularization 
    layers.BatchNormalization(),
    layers.Dense(units=num_classes, activation='softmax')
])
keras.backend.clear_session()
model.summary()

## Compile model
- Cross-entropy is the default loss function to use for multi-class classification problems. ~ [Link](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/#:~:text=Cross%2Dentropy%20is%20the%20default%20loss%20function%20to%20use%20for%20multi%2Dclass%20classification%20problems.)
    - IMPORTANT: The function requires that the output layer is configured with an n nodes (one for each class), in this case three nodes, and a ‘softmax‘ activation in order to predict the probability for each class.

In [ ]:
# Configure optimizer
opt = 

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', # standard for multi-class clf    
    metrics=['accuracy']
)

### Plot the model architecture

In [ ]:
dot_img_file = '/tmp/'+model.name+'.png'
model_img = tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)
#model_img

## Train model

In [ ]:
EPOCHS = 15
STEPS = int(len(train_data_pf))
VALIDATION_STEPS = int(len(valid_data_pf)*0.1)

header=f"|Epochs: {EPOCHS} | Steps: {STEPS} | Validation steps: {VALIDATION_STEPS}"
stars = "\n"+("*"*len(header)); print(header, stars)

history = model.fit(
    train_data_pf,
    validation_data=valid_data_pf,
    validation_steps=10, # at the end of each epoch
    epochs=EPOCHS,
    workers=-1, verbose=1, callbacks=[
        tf.keras.callbacks.EarlyStopping( 
            #Prevent overfitting through early stopping
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
)])
model.save("birds.h5")

In [ ]:
# Evaluation
scores = model.evaluate(test_data_pf, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

# 4. Train model

## Training Results
### Visualize Training

In [ ]:
def plotHistory(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    #plt.plot(epochs, loss, 'g', label='Training loss')
    #plt.plot(epochs, val_loss, 'o', label='Validation loss')
    
    plt.title('Training and validation accuracy and loss')
    plt.legend(loc=0)
    plt.figure()

    plt.show()

plotHistory(history)

In [ ]:
def plotHistory2(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(EPOCHS)

    plt.figure(figsize=(16, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend()#loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend()#loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
plotHistory2(history)

# 5. Evaluation

In [ ]:
scores = model.evaluate(test_data_pf, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

---
# README
after the first meeting

### Aufteilung
- keras tensorflow - Clemens
- pytorch - Jakob
- PCA + preprocessing - Lena 


### Methoden 
- pca?
- image segementation
- Wie laden wir die Bilder von der CSV ins Notebook?
- Wieviele Datenreihen brauchen wir? 
data set
| train    58388 | test      2000 | valid     2000 |


**Take aways from [Gabriel Atkin's Age Prediction From Facial Images](https://www.youtube.com/watch?v=9AnCNBL8c6Q&t=661s):**
- Recurrent feature extraction
- Flatten layer
    - layers.Flatten()(x) # sometimes too many features
    - layers.GlobalAveragePooling2D()(x) # average across the first 2 dimensions

"""    # convolution layer
    layers.Conv2D(16, (3,3), activation='relu', input_shape=INPUT_SHAPE),
    layers.MaxPooling2D(),
    
    # convolution layer
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    # convolution layer
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    # convolution layer
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),"""

### Links:
- TF model.fit() -> https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
- ...

**Optimizers:**
- Adam: a sensible default optimizer
- Nadam (Nesterov-accelerated Adam)
- RMSProp (oftentimes used for regression) -> keras default

# Links
## Good Notebook for reference
### https://www.kaggle.com/code/ashwinshetgaonkar/bird-classifier-tensorflow-beginner

## Learn TensorFlow in this notebook
### http://bit.ly/2lXXdw5